# TVS Hackathon Project: 


## run maf

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
#import seaborn as sn

import healpy as hp
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db

In [ ]:
# connect to opsim database
opsdb_baseline = db.OpsimDatabase('opsdb/baseline2018a.db')
# opsdb_pontus = db.OpsimDatabase('opsdb/pontus_2573.db')

# output directory
outDir = 'outdir'
resultsDb = db.ResultsDb(outDir=outDir)

In [ ]:
# run the metric, check same filename in folder
metric = METRICNAME()

slicer = slicers.HealpixSlicer(nside=16) # for skymap
# slicer = slicers.UserPointsSlicer(ra, dec) # for point fields
sqlconstraint = 'night<700 and proposalId=3'

metricSky = metricBundles.MetricBundle(metric,slicer,sqlconstraint)

group = metricBundles.MetricBundleGroup({'metricSky':metricSky}, opsdb, outDir=outDir, resultsDb=resultsDb)
group.runAll()
group.plotAll(closefigs=False)

## write a new metric

In [ ]:
class newMetric(metrics.BaseMetric):
    '''the simplest new metric'''
    
    def __init__(self, colname, **kwargs):
        self.colname = colname
        super().__init__(col=colname,**kwargs)
        
        # use dataout to set different output type
        #if self.dataout:
        #    super().__init__(col=self.colname, metricDtype='object', **kwargs)
        #else:
        #    super().__init__(col=self.colname, metricDtype='float', **kwargs)
        
    def run(self, dataSlice, slicePoint=None):
        '''method used to calculate colume data, return a values for each slicepoint'''
        
        result = np.sum(dataSlice[self.colname])  
        return result

## get dataSlice for test

In [ ]:
class getDataMetric(BaseMetric):
    """
    returns required data 
    """
    
    def __init__(self, colname=['airmass', 'expMJD'], dataout=True, returnIdx=1, **kwargs):
        self.colname = colname
        self.returnIdx = returnIdx
        self.dataout = dataout
        if self.dataout:
            super().__init__(col=self.colname, metricDtype='object', **kwargs)
        else:
            super().__init__(col=self.colname, metricDtype='float', **kwargs)
        
        
    def run(self, dataSlice, slicePoint=None):
        
        # return dataSlice directly
        if self.dataout:
            result = dataSlice
            return result
        else:
            result = np.mean(dataSlice[self.colname[int(self.returnIdx)]])
            return float(result)


## utils

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# plot light curve
def plotlc(dataSlice, time_key='time', mag_key='mag', filter_key='filter', err_key = None,
          peakEpoch=None, nearPeakT=None, postPeakT=None):
    # Input: dataSlice, pandas DataFrame or numpy ndarray
    import numpy as np

    colors = {'u':'k', 'g':'b', 'r':'r', 'i':'m', 'z':'orange', 'y':'c'}
    
    plt.figure(figsize=(10, 6))
    for f in np.unique(dataSlice[filter_key]):
        fmatch = dataSlice[filter_key]==f
        time = dataSlice[time_key][fmatch]
        mag = dataSlice[mag_key][fmatch]
        
        # draw a vertical line to denote pre/near/post peak
        if peakEpoch!=None:
            plt.axvline(x=peakEpoch, linestyle='-', linewidth=1)
            plt.axvline(x=peakEpoch-nearPeakT/2, linestyle='--', linewidth=1)
            plt.axvline(x=peakEpoch+nearPeakT/2, linestyle='--', linewidth=1)
            plt.axvline(x=peakEpoch+nearPeakT/2+postPeakT, linestyle='--', linewidth=1)
        
        if err_key!=None:
            mag_err = dataSlice[err_key][fmatch]
            plt.errorbar(time, mag, yerr=mag_err, color=colors[f], fmt='.', label=f)
        else:
            plt.scatter(time, mag, color=colors[f], marker='*', label=f)

    plt.legend()
    plt.xlabel(time_key)
    plt.ylabel(mag_key)
    plt.ylim(plt.ylim()[::-1])

# plot skymap
def plot_mwd(RA, Dec, c, org=0, title='Mollweide projection', projection='mollweide'):
    ''' RA, Dec are arrays of the same length.
    RA takes values in [0,360), Dec in [-90,90],
    which represent angles in degrees.
    org is the origin of the plot, 0 or a multiple of 30 degrees in [0,360).
    title is the title of the figure.
    projection is the kind of projection: 'mollweide', 'aitoff', 'hammer', 'lambert'
    
    Examples:
    >>> plot_mwd(df['ra'], df['dec'], df['metricValues'])
    '''
    x = np.remainder(RA+360-org,360) # shift RA values
    ind = x>180
    x[ind] -=360    # scale conversion to [-180, 180]
    x=-x    # reverse the scale: East to the left
    tick_labels = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
    tick_labels = np.remainder(tick_labels+360+org,360)
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(111, projection=projection, facecolor ='LightCyan')
    sc = ax.scatter(np.radians(x), np.radians(Dec), c=c, marker='.')  # convert degrees to radians
    ax.set_xticklabels(tick_labels)     # we add the scale on the x axis
    ax.set_title(title)
    ax.title.set_fontsize(15)
    ax.set_xlabel("RA")
    ax.xaxis.label.set_fontsize(12)
    ax.set_ylabel("Dec")
    ax.yaxis.label.set_fontsize(12)
    ax.grid(True)
    plt.colorbar(sc)

# convert between mag and flux
def mag2flux(m, m0=18):
    return 10**(-0.4 * (m - m0))

def flux2mag(fx, m0=18):
    return -2.5 * np.log10(fx) + m0

# convert between SNR and standard deviation
def snr2std(self, snr):
    # standard deviation of magnitudes.
    std = 2.5 * np.log10(1 + 1/snr)
    return std 

# units convertion
from astropy.coordinates import ICRS, Galactic
from astropy import units as u
def equ2gal(ra, dec):
    # convert from equ to gal
    c = ICRS(ra=ra*u.degree, dec=dec*u.degree)
    g = c.transform_to(Galactic)
    return g.l.deg, g.b.deg

def gal2equ(l, b):
    '''convert from l,b to ra, dec, in degree'''
    c = Galactic(l=l*u.degree, b=b*u.degree)
    eq = c.transform_to(ICRS)
    return eq.ra.deg, eq.dec.deg

